#  Models

## Imports 

In [ ]:
#Directory Libraries
import sys, os
import glob
from os.path import join
#plotting libraries
import matplotlib.pylab as plt
from tqdm.notebook import tqdm
import seaborn as sns
#pre-processing 
import cv2
from numpy.random import seed
import pandas as pd
from PIL import Image
import numpy as np
#tensorflow
import tensorflow
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping, CSVLogger

from tensorflow.keras.applications import Xception
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow.keras.backend as K
from tensorflow.keras import layers,Model
from tensorflow.keras import initializers, layers
from tensorflow.keras.applications.vgg19 import VGG19
from tensorflow.keras import layers, models, optimizers, Input
from tensorflow.keras import backend as K
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam
#tensorflow.device("GPU:0")
#parallel computing
from joblib import Parallel, delayed
import argparse
#Ignore unwanted warnings
import warnings
warnings.filterwarnings('ignore')
seed(101)
%matplotlib inline
#Make sure that plots are colorblindfriendly
plt.style.use("seaborn-colorblind")

#metrices 
# demonstration of calculating metrics for a neural network model using sklearn
from sklearn.datasets import make_circles
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import confusion_matrix

In [ ]:
Processed_datasets = {
    'train_dir': '../input/capstone-dataset/base__dir/train_dir',
    'val_dir': '../input/capstone-dataset/base__dir/val_dir',
    'test_dir':'../input/capstone-dataset/base__dir/test_dir'
    
}


In [ ]:
df_train = pd.read_csv("../input/capstone-dataset/base__dir/train_data.csv")
df_val = pd.read_csv("../input/capstone-dataset/base__dir/val_data.csv")
df_test = pd.read_csv("../input/capstone-dataset/base__dir/test_data.csv")


In [ ]:
num_train_samples = len(df_train)
num_val_samples = len(df_val)
num_test_samples = len(df_test)

train_batch_size = 10
val_batch_size = 10
test_batch_size = 10

train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)
test_steps = np.ceil(num_test_samples / test_batch_size)


In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255)
test_datagen = ImageDataGenerator(rescale=1./255)


train_gen = datagen.flow_from_directory(Processed_datasets['train_dir'],
                                        target_size=(150,150),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = test_datagen.flow_from_directory(Processed_datasets['val_dir'],
                                        target_size=(150,150),
                                        batch_size=val_batch_size,
                                        class_mode='categorical',shuffle=False)
test_gen = test_datagen.flow_from_directory(Processed_datasets['test_dir'],
                                        target_size=(150,150),
                                        batch_size=test_batch_size,
                                        class_mode='categorical',shuffle=False)

In [ ]:
odel = Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape=(150, 150, 3),
    include_top=False,pooling ='avg')  # Do not include the ImageNet classifier at the top.
inputs = Input(shape=(150, 150, 3))

for layer in base_model.layers:
    layer.trainable = True
mode_version = "Xception"
checkpoint = callbacks.ModelCheckpoint(f'weights-{mode_version}', monitor='val_accuracy', save_best_only=True, save_weights_only=True, verbose=1)

early_stopper = EarlyStopping(monitor ="val_accuracy", patience=5)
x = base_model.output
droup_out = Dropout(0.3)(x)
predicted = Dense(2,activation ='softmax')(droup_out)
model_pretrain = Model(base_model.input, predicted)

model_pretrain.compile(loss=tensorflow.keras.losses.categorical_crossentropy,
          optimizer=Adam(lr = 0.0005),
          metrics=['accuracy'])
pretraining_Xception =model_pretrain.fit(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=30,callbacks =[checkpoint,early_stopper])


In [ ]:
pretraining_Xception_history = pd.DataFrame(pretraining_Xception.history)
pretraining_Xception_history.to_csv("Xception.history.csv")


In [ ]:
predict = model_pretrain.predict_generator(test_gen,steps = test_steps)


In [ ]:
 model_pretrain.evaluate(test_gen,
                            steps = test_steps)

### Get the Metrices for the Test and the Validation

In [ ]:
# predict probabilities for test set
yhat_probs = model_pretrain.predict_generator(test_gen,
                            steps = test_steps)
# predict crisp classes for test set
yhat_classes = np.argmax(yhat_probs,axis=1)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
 
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(test_gen.classes, yhat_classes)
print('Test Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(test_gen.classes, yhat_classes)
print(' TestPrecision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(test_gen.classes, yhat_classes)
print('Test Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(test_gen.classes, yhat_classes)
print('Test F1 score: %f' % f1)
 
# kappa

# ROC AUC
auc = roc_auc_score(test_gen.classes, yhat_probs)
print('Test ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(test_gen.classes, yhat_classes)
print(matrix)

In [ ]:
 
# predict probabilities for test set
yhat_probs = model_pretrain.predict_generator(val_gen,
                            steps = val_steps)
# predict crisp classes for test set
yhat_classes = np.argmax(yhat_probs,axis=1)
# reduce to 1d array
yhat_probs = yhat_probs[:, 0]
 
# accuracy: (tp + tn) / (p + n)
accuracy = accuracy_score(val_gen.classes, yhat_classes)
print('valid Accuracy: %f' % accuracy)
# precision tp / (tp + fp)
precision = precision_score(val_gen.classes, yhat_classes)
print(' Tvalid Precision: %f' % precision)
# recall: tp / (tp + fn)
recall = recall_score(val_gen.classes, yhat_classes)
print('valid Recall: %f' % recall)
# f1: 2 tp / (2 tp + fp + fn)
f1 = f1_score(val_gen.classes, yhat_classes)
print('valid F1 score: %f' % f1)
 
# kappa

# ROC AUC
auc = roc_auc_score(val_gen.classes, yhat_probs)
print('valid ROC AUC: %f' % auc)
# confusion matrix
matrix = confusion_matrix(val_gen.classes, yhat_classes)
print(matrix)